In [1]:
using ProgressMeter
using Base.Test

In [2]:
using ParserCombinator
using NamedColors
using Colors
using AutoHashEquals
using AbstractTrees

In [3]:
@with_names begin
    athing = Delayed()
    thing = Repeat(p"\w") |> join
    things = athing + E" " + athing
    athing.matcher = thing | things
    tgrammer = athing + Eos()
end

ParserCombinator.Seq(:tgrammer,ParserCombinator.Matcher[ParserCombinator.Delayed(:athing,ParserCombinator.Alt(:Alt,ParserCombinator.Matcher[ParserCombinator.Transform(:thing,ParserCombinator.Depth(:Depth,ParserCombinator.Pattern(:Pattern,"\\w",r"^(?:\w)(.??)",()),0,9223372036854775807,true),ParserCombinator.#50),ParserCombinator.Seq(:things,ParserCombinator.Matcher[ParserCombinator.Delayed(#= circular reference @-5 =#),ParserCombinator.Drop(:Drop,ParserCombinator.Equal(:Equal," ")),ParserCombinator.Delayed(#= circular reference @-5 =#)])])),ParserCombinator.Eos(:Eos)])

In [4]:
parse_dbg("a", tgrammer)

1-element Array{Any,1}:
 "a"

In [5]:
start("aa")

1

In [6]:
next("abcde",1)

('a',2)

In [7]:
function common_prefix_key(strings::Trie, source)
    Task() do 
        node = strings
        ii = start(source)
        while !done(source, ii)
            s, next_ii = next(source, ii)
            if haskey(node.children, s)
                node = node.children[s]
                if node.is_key
                    produce(ii)
                end
                ii=next_ii
            else
                break
            end
        end
    end
end

LoadError: UndefVarError: Trie not defined

In [8]:
strings = Trie(["abc", "aabbc", "aabbcc", "aaabc", "aax"]);

common_prefix_key(strings, "aa") |> collect

LoadError: UndefVarError: Trie not defined

In [9]:
function check_prefix_key(
        strings::Trie,
        source::AbstractString;
        longest::Bool = true)
    node = strings
    foundat = -1
    
    function cknode!(ii)
        if node.is_key
            foundat = ii
        end
        node.is_key
    end

    for (ii, s) in zip(0:length(source), source)
        if haskey(node.children, s)
            node = node.children[s]
            if cknode!(ii+1) && !longest
                # if we are looking for shortest then we found it.
                break 
            end
        else
            break
        end
    end
    foundat <= 0 ?  Nullable{String}() : Nullable(source[1:foundat])
end

LoadError: UndefVarError: Trie not defined

In [10]:
@testset "Check for prefix with tries" begin
    strings = Trie(["abc", "aabbc", "aabbcc", "aaabc", "aax"]);

    @test check_prefix_key(strings, "ab"; longest=false) |> isnull
    @test check_prefix_key(strings, "abc"; longest=false) |> get =="abc"
    @test check_prefix_key(strings, "abcx"; longest=false) |> get == "abc"
    @test check_prefix_key(strings, "aabbcc"; longest=false) |> get == "aabbc"
    @test check_prefix_key(strings, "aabbcx"; longest=false) |> get == "aabbc"


    @test check_prefix_key(strings, "ab") |> isnull
    @test check_prefix_key(strings, "abc") |> get =="abc"
    @test check_prefix_key(strings, "abcx") |> get == "abc"
    @test check_prefix_key(strings, "aabbcc") |> get == "aabbcc"
    @test check_prefix_key(strings, "aabbcx") |> get == "aabbc"    
end;

Check for prefix with tries: Error During Test
  Got an exception of type UndefVarError outside of a @test
  UndefVarError: Trie not defined
   in macro expansion; at ./In[10]:2 [inlined]
   in macro expansion; at ./test.jl:672 [inlined]
   in anonymous at ./<missing>:?
   in include_string(::String, ::String) at ./loading.jl:441
   in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:175
   in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/eventloop.jl:8
   in (::IJulia.##9#15)() at ./task.jl:360
Test Summary:               | Error  Total
  Check for prefix with tries |     1      1


LoadError: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.

In [11]:
@auto_hash_equals type EqualsOneOf<:Matcher
    name::Symbol
    strings::Trie
    longest::Bool
    function EqualsOneOf(strings; longest=true) 
        new(:EqualsOneOf, Trie(strings), longest)
    end
end

always_print(::EqualsOneOf) = true

function execute(k::Config, m::EqualsOneOf, s::Clean, i)
    for x in m.string
        if done(k.source, i)
            return FAILURE
        end
        y, i = next(k.source, i)
        if x != y
            return FAILURE
        end
    end
    Success(DIRTY, i, Any[m.string])
end

LoadError: UndefVarError: Trie not defined

In [12]:
using DataStructures

In [13]:
"""
Digests a set of raw color strings,
and finds lists of the "Parts Of Colors" (poc), similar to Parts of speech (pos).
These are

- `base_colors`: Normal colors people use all the time, eg `red`, `green`, `tan`, `purple`
- `generic_modifiers`: Adjectives that people apply to colors, `dark`, `pale` etc. Also includes (for POC tagging purposes) things like `greenish` and `bluey`. Though they should be parsed differently in next step 
- `nounish_colors`: Things that are not normally considered colors but use like them anyway (and don't thus normally occur with genric modifiers), eg `blood`, `puke`, `booger`
- `nounish_modifiers`: Nouns that are attached to specific colors, that indicate a specific type of that color.  Eg grass as in "grass green", "ghost" as in "ghost white"
- `conjunctives`: Words that go between two colors to join them. eg "with".

In general be confidant on the identification of all `base_colors` and `generic_modifiers`,
if capturing from a large number of `raw_color_names`
Be less confidant of the indification of `nounish_colors` and `nounish_modifiers`
have no confidance in the identifaction of `conjuctives, they are for debugging purposes
"""
function find_POCs(raw_color_names, nounish_cutoff = 2)
    split_on = [' ','-','/']
    first_color_words = first.(split.(raw_color_names, [split_on]))
    last_color_words =  last.(split.(raw_color_names,  [split_on]))

    first_color_counts = counter(first_color_words)
    last_color_counts = counter(last_color_words)

    base_colors = String[] #Generic colors
    nounish_colors = String[] #Words that are used as colors but probable are not normally recognised as suc

    modifiers = String[] #Adjectives that modify a color
    nounish_modifiers = String[] #Noun parts that describe a paricular color, eg lawn ad in lawn green


    for name in union(first_color_words, last_color_words)
        fcount = first_color_counts[name] 
        lcount = last_color_counts[name]
        if fcount  <= lcount 
            #including On a tie
            if fcount <= nounish_cutoff
                push!(nounish_colors, name)
            else
                push!(base_colors, name)
            end
        else
            if fcount <= nounish_cutoff
                push!(nounish_modifiers, name)
            else
                push!(modifiers, name)
            end
        end
    end
    
    mid(x) =  length(x)>2 ? x[2:end-1]: []
    conjunctives = Base.flatten(mid.(split.(raw_color_names, [split_on]))) |> Set 
    setdiff!(conjunctives, first_color_words)
    setdiff!(conjunctives, last_color_words) #remove anything already categorised
    
    
    base_colors, modifiers, nounish_colors, nounish_modifiers, conjunctives
end

find_POCs

In [14]:
(base_colors, 
modifiers,
nounish_colors,
nounish_modifiers,
conjunctives) = find_POCs(collect(keys(load_xkcd())))


(String["purple","blue","turquoise","grey","violet","aqua","brown","orange","green","tan","yellow","rose","lime","red","olive","seafoam","pink","lavender","teal","navy"],String["dark","light","mud","greenish","pale","pea","slate","deep","bluey","faded"  …  "bluish","very","purplish","off","purpleish","shit","golden","poop","cool","steel"],String["celery","chocolate","heliotrope","puce","scarlet","copper","ocher","liliac","rouge","custard"  …  "heather","sandstone","lavendar","siena","cotta","soup","rod","urple","cheese","shell"],String["grass","lipstick","pine","twilight","cornflower","pumpkin","cerulean","carnation","clay","jade"  …  "eggshell","night","sunny","indian","clear","stormy","sunshine","murky","key","hospital"],Set(Any["a","and","out","gum","with","of","engine","hint"]))

In [15]:
using ParserCombinator


abstract AbstractColorBit
abstract AbstractModifier<:AbstractColorBit

abstract AbstractColorDesc<:AbstractColorBit
abstract AbstractColorName<:AbstractColorDesc
abstract AbstractCombinedColor<:AbstractColorDesc


@auto_hash_equals immutable BaseColorName{S<:AbstractString} <:AbstractColorName
    val::S
end

@auto_hash_equals immutable NounishColorName{S<:AbstractString} <:AbstractColorName
    val::S
end

@auto_hash_equals immutable GenericModifier{S<:AbstractString} <:AbstractModifier 
    val::S 
end

@auto_hash_equals immutable NounishModifier{S<:AbstractString} <:AbstractModifier 
    val::S 
end

@auto_hash_equals immutable PostFixedColorModifier{
        S<:AbstractString,
        V<:AbstractColorDesc}<:AbstractModifier
    val::V
    suf::S
end

@auto_hash_equals immutable BinaryOpedColor{
        S<:AbstractString,
        V1<:AbstractColorDesc,
        V2<:AbstractColorDesc} <:AbstractCombinedColor 
    val1::V1
    op::S
    val2::V2
end

@auto_hash_equals immutable ModifiedColor{
        S<:AbstractModifier, 
        V<:AbstractColorDesc}<:AbstractCombinedColor
    mod::S
    val::V
end

@auto_hash_equals immutable PostFixedColor{
        S<:AbstractString,
        V<:AbstractColorDesc}<:AbstractCombinedColor
    val::V
    suf::S
end

In [16]:
function AbstractTrees.children{T<:AbstractColorBit}(v::T)
    fnames = fieldnames(T)
    getfield.([v], fnames)
end

function AbstractTrees.printnode{T<:AbstractColorBit}(io::IO, ::T)
    showcompact(io,T.name)
end

In [48]:
@with_names begin

    #word = Repeat(p"\w"; greedy=false, backtrack=true) |> join #can't use regex `p"\w+"` as it has no bactracking
    #word = Repeat(Alt(Equal.('a':'z')...)) |> join
    word = p"\w+"
    
    col  = Alt(Equal.(base_colors)...) >  BaseColorName
    nmod = word > NounishModifier
    ncol = word > NounishColorName
    gmod  = Alt(Equal.(modifiers)...)  > GenericModifier


    color = Delayed()
    binop = e" "#(e" with a hint of " | e"ish " 
            #| e"y " | e"/" | e"-")

    binaryoped = color + binop + color > BinaryOpedColor

    postfix = e"y" | e"ish" | e""
    postfixed = color + postfix > PostFixedColor
    
    pmod = col + postfix > PostFixedColorModifier
    mod = pmod | gmod | nmod
    modded = mod +  E" " + color > ModifiedColor
    

    color.matcher = (col | modded | ncol)
    colorgrammar = color + Eos();
end
""


""

In [49]:
t = parse_one("blueish redish green", (colorgrammar); delegate=Cache)
print_tree(STDOUT, t)

Any[ModifiedColor{PostFixedColorModifier{String,BaseColorName{String}},ModifiedColor{PostFixedColorModifier{String,BaseColorName{String}},BaseColorName{String}}}(PostFixedColorModifier{String,BaseColorName{String}}(BaseColorName{String}("blue"),"ish"),ModifiedColor{PostFixedColorModifier{String,BaseColorName{String}},BaseColorName{String}}(PostFixedColorModifier{String,BaseColorName{String}}(BaseColorName{String}("red"),"ish"),BaseColorName{String}("green")))]
└─ ModifiedColor
   ├─ PostFixedColorModifier
   │  ├─ BaseColorName
   │  │  └─ "blue"
   │  └─ "ish"
   └─ ModifiedColor
      ├─ PostFixedColorModifier
      │  ├─ BaseColorName
      │  │  └─ "red"
      │  └─ "ish"
      └─ BaseColorName
         └─ "green"


In [52]:
t = parse_one("gob rat", colorgrammar; delegate=Cache)
print_tree(STDOUT, t)

Any[ModifiedColor{NounishModifier{SubString{String}},NounishColorName{SubString{String}}}(NounishModifier{SubString{String}}("gob"),NounishColorName{SubString{String}}("rat"))]
└─ ModifiedColor
   ├─ NounishModifier
   │  └─ "gob"
   └─ NounishColorName
      └─ "rat"


In [56]:
parse_one("Grass Green", colorgrammar)

1-element Array{Any,1}:
 ModifiedColor{NounishModifier{SubString{String}},NounishColorName{SubString{String}}}(NounishModifier{SubString{String}}("Grass"),NounishColorName{SubString{String}}("Green"))

In [22]:
using Base.Test
@test parse_one("blue", colorgrammar) |> first == BaseColorName("blue")
@test parse_one("redish blue", colorgrammar) |> first ==
    BinaryOpedColor(BaseColorName("red"),"ish ",BaseColorName("blue"))
@test parse_one("redish green-red", colorgrammar) |> first ==
BinaryOpedColor(BaseColorName("red"),"ish ", BinaryOpedColor(BaseColorName("green"), "-", BaseColorName("red")))

Test Failed
  Expression: parse_one("redish blue",colorgrammar) |> first == BinaryOpedColor(BaseColorName("red"),"ish ",BaseColorName("blue"))
   Evaluated: ModifiedColor{PostFixedColorModifier{String,BaseColorName{String}},BaseColorName{String}}(PostFixedColorModifier{String,BaseColorName{String}}(BaseColorName{String}("red"),"ish"),BaseColorName{String}("blue")) == BinaryOpedColor{String,BaseColorName{String},BaseColorName{String}}(BaseColorName{String}("red"),"ish ",BaseColorName{String}("blue"))


LoadError: There was an error during testing

In [23]:
t=
print_tree(STDOUT, t)

Any[ModifiedColor{PostFixedColorModifier{String,BaseColorName{String}},BaseColorName{String}}(PostFixedColorModifier{String,BaseColorName{String}}(BaseColorName{String}("green"),"ish"),BaseColorName{String}("red"))]
└─ ModifiedColor
   ├─ PostFixedColorModifier
   │  ├─ BaseColorName
   │  │  └─ "green"
   │  └─ "ish"
   └─ BaseColorName
      └─ "red"


In [24]:
parse_one("a", e"a" + Eos() + Eos())

1-element Array{Any,1}:
 "a"

In [25]:
@showprogress for colorstring in collect(keys(load_xkcd()))
    println(colorstring)
    flush(STDOUT)
    parse_one(colorstring, colorgrammar)
end

grass green
Progress:   0%|                                         |  ETA: 0:13:51lipstick


LoadError: InterruptException:

In [26]:
children(BaseColorName("red"))

1-element Array{String,1}:
 "red"

In [27]:
T = typeof(t[1])

LoadError: MethodError: no method matching getindex(::Void, ::Int64)

In [28]:
T.name

LoadError: UndefVarError: T not defined